<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/033_Agent_Loop_with_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s break down the differences and why they matter in the evolution of building LLM agents. These are **core design upgrades** that remove a lot of friction from earlier agent systems.

---

### ✅ 1. **No More Custom Parsing Logic**

**Before (Old Way):**

* You had to write fragile logic to force the LLM to return JSON.
* Then you'd try to `json.loads()` the text and hope it worked.
* You might say:
  *“Respond only with a JSON object that includes keys X and Y...”*
  and then try to validate it manually.

**Now (With Function Calling):**

* You define a schema with `tools=[...]`.
* The model returns the tool call in structured format:

  ```json
  {
    "tool_calls": [
      {
        "function": {
          "name": "read_file",
          "arguments": "{\"file_name\": \"example.txt\"}"
        }
      }
    ]
  }
  ```

**✅ What this gives you:**
Reliable, automatic structured output. You no longer treat the LLM like a shaky string generator — it behaves like a typed interface.

---

### ✅ 2. **Dynamic Execution**

**Before:**

* You had to create a big `if/else` logic tree:

  ```python
  if "read" in response and "file" in response:
      call read_file()
  ```

**Now:**

* The model explicitly says:
  *“Use this function with these arguments.”*
* You simply read the output and run it:

  ```python
  tool_name = tool_call.function.name
  args = json.loads(tool_call.function.arguments)
  result = tool_functions[tool_name](**args)
  ```

**✅ What this gives you:**
A flexible system where the LLM controls the flow — it tells you what to do, and you just do it.

---

### ✅ 3. **Unified Text & Action Handling**

**Before:**

* You had to manually guess:
  *“Is the LLM trying to say something or should I run a function?”*

**Now:**

* The OpenAI API cleanly separates this:

  * If it wants to call a function → `tool_calls` is present
  * If not → it just responds with text in `message.content`

**✅ What this gives you:**
A natural **blend of conversation and action**, like:

* “Let me grab the file for you…” → (calls `read_file`)
* “Here are the results…” → (plain response)

You no longer need a separate “chat mode” and “action mode”.

---

### ✅ 4. **Automated Function Execution**

**Before:**

* You wrote boilerplate glue code to match intent to function:

  ```python
  if intent == "get_stock_price":
      get_stock_price(ticker)
  ```

**Now:**

* It’s just:

  ```python
  tool_name = tool_call.function.name
  result = tool_functions[tool_name](**args)
  ```

The mapping is automatic — you use a Python dictionary to connect schema → function.

**✅ What this gives you:**
A **scalable architecture**: add new tools by just:

* Defining the Python function
* Adding a schema entry

No need to rewrite core logic.

---

### 🧠 Final Thought

These changes represent a big leap:

> 🧩 Instead of parsing messy guesses, you’re building structured, reliable agents using a formal interface with the LLM.

That’s a real shift in **trust and power** — and why function calling is a game-changer for agent development.




In [ ]:
%pip install -qU dotenv openai

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re
import textwrap
from typing import List
from litellm import completion

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

def terminate(message: str) -> None:
    """Terminate the agent loop and provide a summary message."""
    print(f"Termination message: {message}")

tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},
                },
                "required": ["message"]
            }
        }
    }
]

agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:

    messages = agent_rules + memory

    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    if response.choices[0].message.tool_calls:
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name
        tool_args = json.loads(tool.function.arguments)

        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        if tool_name == "terminate":
            print(f"Termination message: {tool_args['message']}")
            break
        elif tool_name in tool_functions:
            try:
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:
                result = {"error":f"Error executing {tool_name}: {str(e)}"}
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},
            {"role": "user", "content": json.dumps(result)}
        ])
    else:
        result = response.choices[0].message.content
        print(f"Response: {result}")
        break

Here's how to walk through and learn from the code, along with **key parts you should focus on** as you dissect this notebook:

---

### ✅ **Big Picture: What's Different Here?**

This lecture introduces a **function-calling agent loop** that is:

* **Robust**: Tools return structured responses using OpenAI’s built-in function-calling format.
* **Simplified**: No need to parse text or force output structure.
* **Autonomous**: The loop continues until the LLM explicitly uses a `terminate` tool.
* **Flexible**: Easily extendable with new tools.

---

### 🧠 **Step-by-Step Walkthrough and What to Learn**

---

#### 1. **Core Tools / Functions**

```python
def list_files() -> List[str]: ...
def read_file(file_name: str) -> str: ...
def terminate(message: str) -> None: ...
```

✔️ These are the **real Python functions** that do the work.

👉 Learn:

* How the functions relate to the tasks the LLM will ask to perform.
* That `terminate()` doesn't return anything—it's just for flow control.

---

#### 2. **Tool Schema Definitions**

```python
tools = [
  {
    "type": "function",
    "function": {
      "name": "list_files",
      "description": "...",
      "parameters": {...}
    }
  },
  ...
]
```

✔️ Each tool schema tells the LLM:

* What it’s called
* What it does
* What input it expects (in JSON Schema format)

👉 Learn:

* The **strict format** required: `type`, `function`, `parameters`
* `required` is key—OpenAI will enforce that fields are present

---

#### 3. **Agent Rules (System Prompt)**

```python
agent_rules = [{
  "role": "system",
  "content": "You are an AI agent that can perform tasks..."
}]
```

✔️ This primes the model to:

* Know its tools
* Understand the flow (e.g., list before read)
* Use `terminate` at the end

👉 Learn:

* The importance of **clear, high-level behavioral guidance**
* You can write agent policies here

---

#### 4. **Agent Memory + Loop**

```python
memory = [{"role": "user", "content": user_task}]
...
while iterations < max_iterations:
    messages = agent_rules + memory
    response = completion(...)
```

✔️ This loop:

* Keeps state across tool calls
* Appends each tool action and result as chat messages

👉 Learn:

* Memory management: you’re simulating conversation history
* `tool_calls` vs `content`: always check for tool calls before responding

---

#### 5. **Tool Execution**

```python
tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
...
result = tool_functions[tool_name](**tool_args)
```

✔️ The LLM’s response tells you:

* What tool to call
* What arguments to pass

👉 Learn:

* You **don’t need to parse anything manually** anymore
* Just use `.tool_calls` and let the model handle the structure

---

#### 6. **Termination Logic**

```python
if tool_name == "terminate":
    print(...)
    break
```

✔️ Model controls flow. Agent ends when model says to terminate.

👉 Learn:

* This is the backbone of multi-turn autonomous behavior
* You’re trusting the LLM to decide when it’s done

---

#### 7. **Error Handling (Important Takeaway)**

```python
except Exception as e:
    result = {"error": ...}
```

✔️ Prevents crashes and shows feedback.

👉 Learn:

* You still need defensive coding even with structure
* Consider catching `json.JSONDecodeError` too

---

### 🏁 **Key Lessons to Take Away**

| Concept                | Why It Matters                                                  |
| ---------------------- | --------------------------------------------------------------- |
| **Function calling**   | Structured responses from the LLM — no more parsing plain text  |
| **Schema + functions** | You define what’s possible, the LLM chooses what to use         |
| **Agent loop**         | Clean loop that listens, interprets, acts, and learns           |
| **Tool memory**        | Maintaining tool/action history is essential for context        |
| **Terminate tool**     | Gives the model control to stop — huge for autonomy             |
| **Error safety**       | Even smart agents need safety nets for bad inputs or edge cases |




Yes — **you absolutely could build an agent that returns both a tool call *and* a message**, but you'd need to **structure the logic across multiple steps** using the OpenAI Chat Completions API. Here's how it works under the hood and how you’d implement it.

---

### ✅ What You *Can* Do: Simulate Both via a Two-Step Agent Loop

Even though **a single assistant message cannot return both**, you can **design an agent loop** that achieves the effect.

#### 🧩 Step-by-step breakdown:

1. **User asks:**

   > “What’s the current price of AAPL and should I be worried about recent news?”

2. **LLM decides:**
   It needs to call a tool first:

   ```json
   {
     "tool_calls": [
       {
         "function": {
           "name": "get_stock_price",
           "arguments": "{\"ticker\": \"AAPL\"}"
         }
       }
     ]
   }
   ```

3. **Your code executes the tool**, then adds that output as a `tool` role message.

4. **Send the updated `messages` list back** to the LLM.

5. **Now the LLM responds:**

   > “AAPL is currently trading at \$187.65. Based on recent headlines, there’s some volatility due to earnings season. Would you like me to check recent news as well?”

---

### ✅ What You’re Building: A Conversational Agent Loop

The logic looks like this:

```python
while True:
    response = client.chat.completions.create(...)

    if response contains tool_call:
        execute tool
        add tool result to messages
    elif response contains text:
        show final reply
        break
```

That loop allows:

* Multiple tool calls
* Conversational context
* Dynamic back-and-forth reasoning

---

### 🔁 Optional: Multi-tool chaining

Advanced agents (like the one you're building!) can also:

* Detect the need to call **several tools** before responding
* Re-enter the loop until all needed data is gathered

---

### 🧠 Summary

| Goal                                    | Supported?         | How to do it                        |
| --------------------------------------- | ------------------ | ----------------------------------- |
| Tool call + natural response (together) | ❌ Not in 1 message |                                     |
| Tool + response sequence                | ✅ Yes              | Multi-step loop using `tool` role   |
| Multi-tool planning                     | ✅ Yes              | Chain calls using `tool_calls` loop |


